In [4]:
install.packages("comprehenr")

Error in download.file(url, destfile, method, mode = "wb", ...) : 
  ignoring SIGPIPE signal


Warning message in download.packages(pkgs, destdir = tmpd, available = available, :
“download of package ‘comprehenr’ failed”


In [5]:
library(tidyverse)
library(bipartite)
library(comprehenr)
library(stringr)
library(parallel)
library(pbmcapply)
source("/groups/itay_mayrose/halabikeren/tmp/plant_pollinator_inter/feature_computation/utils.R")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.3      ✔ forcats 0.5.2 

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: vegan

Loading required package: permute

Loading required package: lattice

This is vegan 2.6-4

Loading required package: sna

Loading required package: statnet.common


Attaching package: ‘statnet.common’


The following objects are masked from ‘package:base’:

    attr, order


Loading required package: network


‘network’ 1.17.1 (2021-06-12), part of the Statnet Project
* ‘news(package="network")’ for changes since last version
* ‘citation("network")’ for citation information
* ‘https://statnet.org’ for help, support, and other information


sna: Tools for Social Netwo

In [11]:
networks_dir = "../../data/networks/all/"
networks_types = c("binary", "weighted", "binarized_weighted")
null_networks_dir = "../../data/networks/null/"
skip=c()
#skip = c('../../data/networks/all/weighted/10.csv', '../../data/networks/all/weighted/11.csv', '../../data/networks/all/weighted/12.csv', '../../data/networks/all/weighted/51.csv', '../../data/networks/all/weighted/190.csv', '../../data/networks/all/weighted/192.csv', '../../data/networks/all/weighted/196.csv', '../../data/networks/all/weighted/588.csv')

In [12]:
dir.create(null_networks_dir)
for (net_type in networks_types)
    {
        dir.create(paste(null_networks_dir, net_type, sep=""))
    }

Warning message in dir.create(null_networks_dir):
“'../../data/networks/null' already exists”
Warning message in dir.create(paste(null_networks_dir, net_type, sep = "")):
“'../../data/networks/null/binary' already exists”
Warning message in dir.create(paste(null_networks_dir, net_type, sep = "")):
“'../../data/networks/null/weighted' already exists”
Warning message in dir.create(paste(null_networks_dir, net_type, sep = "")):
“'../../data/networks/null/binarized_weighted' already exists”


In [13]:
full_network_paths = c()
for (net_type in networks_types)
    {
        net_type_dir = paste(networks_dir, net_type, "/", sep="")
        networks_paths = list.files(net_type_dir) 
        full_network_paths <- append(full_network_paths, to_vec(for(p in networks_paths) paste(net_type_dir, p, sep="")))
    }

In [14]:
length(full_network_paths)

[1] 1257

In [11]:
generate_null_networks <- function(path, nsim=100) 
{
    network = process_network(path) 
    output_dir = str_replace(str_replace(path, networks_dir, null_networks_dir), ".csv", "/")
    dir.create(output_dir, showWarnings = FALSE)
    print(output_dir)
    num_files <- length(list.files(output_dir))
    print(num_files)
    if (num_files == nsim)
    {
        return(0)
    }
    null_networks = r2dtable(n=nsim, r=rowSums(network), c=colSums(network))
    for (i in 1:length(null_networks))
    {
        output_path = paste(output_dir, i, ".csv", sep="")
        null_network = data.frame(null_networks[i])
        colnames(null_network) = colnames(network)
        rownames(null_network) = rownames(network)
        write.csv(null_network, output_path)
    }   
    return(0)
}

In [9]:
i = 0
for (path in full_network_paths)
    {
        print(i)
        res = generate_null_networks(path)
        i = i + 1
    }

In [ ]:
# pbmclapply(X=full_network_paths,
#            FUN=generate_null_networks,
#            mc.cores=30,
#            ignore.interactive=TRUE)